In [6]:
#No86(畳み込みニューラルネットワーク (CNN))
#CNNは基本的に画像認識で使用されることが多い。自然言語処理においては、文をマトリックスに見立てる
import pandas as pd
import re
import numpy as np

# 前処理があまいところがあったので、作成し直す
file = '/content/drive/MyDrive/newsCorpora.csv'
data = pd.read_csv(file, encoding='utf-8', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
data = data.replace('"', "'")
# 特定のpublisherのみ抽出
publishers = ['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']
data = data.loc[data['PUBLISHER'].isin(publishers), ['TITLE', 'CATEGORY']].reset_index(drop=True)

# 前処理
def preprocessing(text):
    text_clean = re.sub(r'[\"\'.,:;\(\)#\|\*\+\!\?#$%&/\]\[\{\}]', '', text)
    text_clean = re.sub('[0-9]+', '0', text_clean)
    text_clean = re.sub('\s-\s', ' ', text_clean)
    return text_clean

data['TITLE'] = data['TITLE'].apply(preprocessing)

# 学習用、検証用、評価用に分割する
from sklearn.model_selection import train_test_split

train, valid_test = train_test_split(data, test_size=0.2, shuffle=True, random_state=64, stratify=data['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=64, stratify=valid_test['CATEGORY'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

# 単語の頻度
from collections import Counter
words = []
for text in train['TITLE']:
  #空文字で区切って、wordに追加していく
    for word in text.rstrip().split():
        words.append(word)
#数える
c = Counter(words)
#辞書の作成
word2id = {}
for i, cnt in enumerate(c.most_common()):
  ##出現頻度2回以上の単語のみ辞書に追加
    if cnt[1] > 1:
        word2id[cnt[0]] = i + 1
# 出現頻度上位10単語
for i, cnt in enumerate(word2id.items()):
    if i >= 10:
        break
    print(cnt[0], cnt[1])
# 単語のID化
def tokenizer(text):
    words = text.rstrip().split()
#単語辞書からそのwordのidを取得．ない場合は0を返す
    return [word2id.get(word, 0) for word in words]
sample = train.at[0, 'TITLE']

0 1
to 2
in 3
on 4
UPDATE 5
The 6
as 7
for 8
To 9
of 10


In [2]:
#modelの可視化
!pip install torchinfo
from torch import nn
import random
import torch
from torch import nn
import torch.utils.data as data
from torchinfo import summary

# 乱数のシードを設定
# parserなどで指定
seed = 1234
random.seed(seed)
np.random.seed(seed)
#再現性を保つ
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
#seed値の作成
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

from gensim.models import KeyedVectors

# 事前学習済みの単語ベクトル
file = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(file, binary=True)

# 学習済み単語ベクトルの取得
VOCAB_SIZE = len(set(word2id.values())) + 2 #辞書のID数 + unknown + パディングID(ID化する単語の種類)
EMB_SIZE = 300 #埋め込みベクトルのサイズ
weights = np.zeros((VOCAB_SIZE, EMB_SIZE)) #学習済み単語ベクトルの初期化
words_in_pretrained = 0
#学習済み単語ベクトルの取得(単語が無いときは正規乱数で初期化)
for i, word in enumerate(word2id.keys()):
    try:
        weights[i] = model[word]
        words_in_pretrained += 1
    except KeyError:
        weights[i] = np.random.normal(scale=0.1, size=(EMB_SIZE,))
weights = torch.from_numpy(weights.astype((np.float32)))

print(f'学習済みベクトル利用単語数: {words_in_pretrained} / {VOCAB_SIZE}')
print(weights.size())

学習済みベクトル利用単語数: 4356 / 4598
torch.Size([4598, 300])


In [3]:
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
Y_train = torch.from_numpy(train['CATEGORY'].map(category_dict).values)
Y_valid = torch.from_numpy(valid['CATEGORY'].map(category_dict).values)
Y_test = torch.from_numpy(test['CATEGORY'].map(category_dict).values)
print(Y_train.size())
print(Y_train)

class NewsDataset(data.Dataset):
    def __init__(self, X, y, phase='train'):
      #X: 単語ベクトルの平均をまとめたテンソル
      #y: カテゴリをラベル化したテンソル
        self.X = X['TITLE']
        self.y = y
        #学習か訓練かを設定する
        self.phase = phase

    def __len__(self):
        #全データサイズを返す
        return len(self.y)

    def __getitem__(self, idx):
        #idxに対応するテンソル形式のデータとラベルを取得
        inputs = torch.tensor(tokenizer(self.X[idx]))
        return inputs, self.y[idx]

train_dataset = NewsDataset(train, Y_train, phase='train')
valid_dataset = NewsDataset(valid, Y_valid, phase='val')
test_dataset = NewsDataset(test, Y_test, phase='val')
idx = 0


torch.Size([3709])
tensor([3, 1, 1,  ..., 2, 0, 0])


In [4]:
#自動的にバッチ内の文章の最大単語数でそろえるようにする。
def collate_fn(batch):
    sequences = [x[0] for x in batch]
    labels = torch.LongTensor([x[1] for x in batch])
    x = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True, padding_value=PADDING_IDX)
    return x, labels

# DataLoaderを作成
batch_size = 64

train_dataloader = data.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, worker_init_fn=seed_worker, generator=g)
valid_dataloader = data.DataLoader(
            valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, worker_init_fn=seed_worker, generator=g)
test_dataloader = data.DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, worker_init_fn=seed_worker, generator=g)

dataloaders_dict = {'train': train_dataloader,
                    'val': valid_dataloader,
                    'test': test_dataloader,}


In [7]:
from torch.nn import functional as F
#CNNクラスの作成
class CNN(nn.Module):
    def __init__(self, vocab_size, emb_size, padding_idx, output_size, out_channels, kernel_heights, stride, padding, emb_weights=None):
        super().__init__()
        if emb_weights != None:  # 指定があれば埋め込み層の重みをemb_weightsで初期化
            self.emb = nn.Embedding.from_pretrained(emb_weights, padding_idx=padding_idx)
        else:
            self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
        self.conv = nn.Conv2d(1, out_channels, (kernel_heights, emb_size), stride, (padding, 0))
        self.drop = nn.Dropout(0.4)
        self.fc = nn.Linear(out_channels, output_size)

    def forward(self, x):
        #テンソルの1階に要素数１の次元を追加する
        emb = self.emb(x).unsqueeze(1)
        #convolution(畳み込み)
        #第1引数はその入力のチャネル数,第2引数は畳み込み後のチャネル数,第3引数は畳み込みをするための正方形フィルタ(カーネル)の1辺のサイズ
        conv = self.conv(emb)
        #要素を削除して、活性化関数(relu)に通す
        act = F.relu(conv.squeeze(3))
        #特徴量をサンプリングして減らす処理
        max_pool = F.max_pool1d(act, act.size()[2])
        #全結合層
        logits = self.fc(self.drop(max_pool.squeeze(2)))
        return logits

# パラメータの設定
VOCAB_SIZE = len(set(word2id.values())) + 2
EMB_SIZE = 300
PADDING_IDX = len(set(word2id.values())) + 1 ##各データの単語数を揃えるために空きを埋めるindex
OUTPUT_SIZE = 4
#出力のチャネル数
OUT_CHANNELS = 100
#カーネルのサイズ
KERNEL_HEIGHTS = 3
#フィルタが移動する幅(基本は１)
STRIDE = 1
#埋め込みあり
PADDING = 1

# モデルの定義
model = CNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE, OUT_CHANNELS, KERNEL_HEIGHTS, STRIDE, PADDING, emb_weights=weights)
x = torch.tensor([tokenizer(sample)], dtype=torch.int64)
print(x)
print(x.size())
print(nn.Softmax(dim=-1)(model(x)))

tensor([[1546, 2903,    0, 2904,  346, 1227,    0,  424,  314,  716]])
torch.Size([1, 10])
tensor([[0.2340, 0.2397, 0.2903, 0.2360]], grad_fn=<SoftmaxBackward0>)
